<a href="https://colab.research.google.com/github/Nick088Official/GPT-SoVITS-Colab/blob/main/GPT_SoVITS_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-SoVITS WebUI

[General GPT-SoVITS Guide](https://rentry.co/GPT-SoVITS-guide#3-initialize-dataset)

Port Made by [Nick088](https://linktr.ee/Nick088)

In [ ]:
#@title Prepare the Environment
#@markdown It git clones the repository.

# get the repo
!git clone https://github.com/RVC-Boss/GPT-SoVITS
%cd GPT-SoVITS

In [ ]:
#@title (OPTIONAL) Mount Google Drive

#@markdown This will create a `GPT-SoVITS` folder in [your Google Drive](https://drive.google.com/drive/u/0/home), that can be used to put things like your model dataset, but you have to **manually put the full path** which can be `/content/drive/MyDrive/GPT-SoVITS/YourDatasetHere`
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the parent directory path
parent_dir = '/content/drive/MyDrive/GPT-SoVITS'

# Create the parent directory if it doesn't exist
if not os.path.exists(parent_dir):
    os.mkdir(parent_dir)

In [ ]:
#@title Install Conda
#@markdown Conda will make the session crash/restart, it's normal, just run the following cells after.
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title Install requirements
#@markdown Will take alot of time..

#@markdown Will install also a fix packages for Chinese ASR model downloaded future on, they need a runtime restart, so the code will restart the runtime too.

from IPython.display import clear_output
import time

%cd GPT-SoVITS
!bash install.sh

# fix packages asr models
!pip uninstall -y cffi modelscope
!pip cache purge
!pip install -q --force-reinstall cffi==1.17.1
!pip install -q --force-reinstall modelscope==1.19.2

clear_output()

print("Installed the requirements!")

time.sleep(5)
# restart runtime
import os
os._exit(0)

In [ ]:
#@title Download Pretrained Models
#@markdown gpt so vits v1 & v2 pretrains + uvr5 weights + chinese asr. (english/japanese asr, faster whisper, should be automatically installed when using it for ASR in the UI)

from IPython.display import clear_output


%cd /content/


# gpt so vits pretrains

!git clone https://huggingface.co/lj1995/GPT-SoVITS/ gptsovits_pretrains_temp

!mv gptsovits_pretrains_temp/* GPT-SoVITS/GPT_SoVITS/pretrained_models/

!rm -rf gptsovits_pretrains_temp


# uvr5 weights

!git clone https://huggingface.co/Delik/uvr5_weights uvr5_weights_temp

!mv uvr5_weights_temp/* GPT-SoVITS/tools/uvr5/uvr5_weights

!rm -rf uvr5_weights_temp


# Chinese ASR models

import os
import shutil
from modelscope.hub.snapshot_download import snapshot_download

model_ids = [
    'damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch',
    'damo/speech_fsmn_vad_zh-cn-16k-common-pytorch',
    'damo/punc_ct-transformer_zh-cn-common-vocab272727-pytorch',
]

target_base_dir = 'GPT-SoVITS/tools/asr/models'

for model_id in model_ids:
    try:
        # Download the model
        cache_path = snapshot_download(model_id)

        # Create a subdirectory for the model
        model_name = model_id.split('/')[-1]
        target_model_dir = os.path.join(target_base_dir, model_name)
        os.makedirs(target_model_dir, exist_ok=True)


        # Move the model files to the model-specific directory
        for filename in os.listdir(cache_path):
            source_path = os.path.join(cache_path, filename)
            destination_path = os.path.join(target_model_dir, filename)
            if os.path.isfile(source_path):
                shutil.move(source_path, destination_path)
            elif os.path.isdir(source_path):
                shutil.move(source_path, destination_path)


        print(f"Model {model_id} downloaded and moved to {target_model_dir}")

    except Exception as e:
        print(f"Error downloading or moving model {model_id}: {e}")


# Clean up (remove the downloaded cache directories) - IMPORTANT!
for model_id in model_ids:
    cache_path = snapshot_download(model_id, cache_dir=".")
    if os.path.exists(cache_path):  # Double-check directory existence
      shutil.rmtree(cache_path, ignore_errors=True)


# English/Japanese ASR should be automatically downloaded.

clear_output()
print("Downloaded!")

In [ ]:
#@title Run UI

#@markdown The first run may take more time as it will download the g2pw model.

#@markdown The type of tunnel you wanna use for seeing the public link, so that if one of them is down, you can use the other one.
Tunnel = "Gradio" #@param ["Gradio", "Ngrok", "Cloudflare", "LocalTunnel"]

#@markdown Also when using Ngrok ,Cloudflare or LocalTunnel, you need to wait for the Local URL to appear, and only after that click on the Public URL which is above. Plus, you have to open UVR in the GPT-SoVITS UI before opening the Public URL.

#@markdown Use the following option only if you chose Ngrok as the Tunnel:

#@markdown You can get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new.


ngrok_tunnel_authtoken = "" #@param {type:"string"}

from IPython.display import clear_output
%cd GPT-SoVITS


if Tunnel == "Gradio":
  %env is_share = True
elif Tunnel == "Ngrok":
  %env is_share = False
  !pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_tunnel_authtoken)
  gptsovits_tunnel = ngrok.connect(9874, bind_tls=True)
  uvr_tunnel = ngrok.connect(9834, bind_tls=True)
  print("GPT-SoVITS Tunnel Public URL:", gptsovits_tunnel.public_url)
  print("UVR Tunnel Public URL:", uvr_tunnel.public_url)
elif Tunnel == "Cloudflare":
  %env is_share = False
  # download cloudflare
  !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  !rm -rf nohup.out
  import time
  # Run cloudflare
  # gpt so vits
  !nohup cloudflared tunnel --url localhost:9874 &
  time.sleep(5)
  # uvr
  !nohup cloudflared tunnel --url localhost:9873 &
  clear_output()
  time.sleep(5)
  # Find and print the Cloudflare URL with a prefix
  cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
  print(f"GPT-SoVITS Tunnel Public URL: {cloudflare_url[0]}")
  print(f"UVR Tunnel Public URL: {cloudflare_url[1]}")
elif Tunnel == "LocalTunnel":
  %env is_share = False
  # install
  !npm install -g localtunnel
  import time
  import urllib
  # run localtunnel
  # gptsovits
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9874 >> url.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  clear_output()

  print(f"GPT-SoVITS Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  # uvr
  with open('url2.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9873 >> url2.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url2.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  print(f"UVR Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  print(f'LocalTunnels Password: {endpoint_ip}')


!python webui.py